# 1. Setup

In [1]:
import networkx as nx
import os
import numpy as np
import math
import torch
from torch import nn
import torch.optim as optim
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

C:\Users\caleb\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Data Preprocessing  
Data Structure:
1. **gList** <Dict>: containing total 31 graphs, which 30 from Synthetic and 1 from youtube,using filename as key  
2. element of gList <Dict>: 'graph':nx.Graph();'score': <Dict> with 'node' and 'score'

In [2]:
# Input data
dpath = ".\\data\\"
gList = dict()
filenames = []

for root, dirs, files in os.walk(dpath):
    for file in files:
        file_path = os.path.join(root, file)
        if 'score' not in file:
            filenames.append(file)
            # Process nodes and edges
            gList[file] = dict()
            gList[file]['graph']=nx.Graph()
            with open(file_path,'r') as f:
                content = f.readlines()
                edges = []
                for line in content:
                    if 'com' not in file:
                        nodes = line[:-1].split('\t')
                    else:
                        #continue # after finish all code run code with com
                        nodes = line[:-1].split(" ")
                    # Create edge tuple and append
                    edges.append((int(nodes[0]),int(nodes[1])))
                gList[file]['graph'].add_edges_from(edges)
                print("{} has {} nodes, {} edges".format(file,gList[file]['graph'].number_of_nodes(),gList[file]['graph'].number_of_edges()))
            
            # Process scores
            scorefile = file.replace(".txt","_score.txt")
            gList[file]['score'] = dict()
            score_file_path = os.path.join(root,scorefile) 
            with open(score_file_path,'r') as f:
                content = f.readlines()
                for line in content:
                    if 'com' not in file:
                        node_score = line[:-1].split('\t')
                        gList[file]['score'][int(node_score[0])] = float(node_score[1])
                    else:
                        #continue # after finish all code run code with com
                        node_score = line[:-1].split()
                        gList[file]['score'][int(node_score[0][:-1])] = float(node_score[1])

0.txt has 5000 nodes, 19982 edges
1.txt has 5000 nodes, 19981 edges
10.txt has 5000 nodes, 19980 edges
11.txt has 5000 nodes, 19983 edges
12.txt has 5000 nodes, 19983 edges
13.txt has 5000 nodes, 19984 edges
14.txt has 5000 nodes, 19982 edges
15.txt has 5000 nodes, 19984 edges
16.txt has 5000 nodes, 19982 edges
17.txt has 5000 nodes, 19981 edges
18.txt has 5000 nodes, 19984 edges
19.txt has 5000 nodes, 19981 edges
2.txt has 5000 nodes, 19980 edges
20.txt has 5000 nodes, 19983 edges
21.txt has 5000 nodes, 19982 edges
22.txt has 5000 nodes, 19982 edges
23.txt has 5000 nodes, 19981 edges
24.txt has 5000 nodes, 19984 edges
25.txt has 5000 nodes, 19982 edges
26.txt has 5000 nodes, 19984 edges
27.txt has 5000 nodes, 19983 edges
28.txt has 5000 nodes, 19982 edges
29.txt has 5000 nodes, 19983 edges
3.txt has 5000 nodes, 19982 edges
4.txt has 5000 nodes, 19984 edges
5.txt has 5000 nodes, 19981 edges
6.txt has 5000 nodes, 19984 edges
7.txt has 5000 nodes, 19983 edges
8.txt has 5000 nodes, 19983 

# 3. DrBC

In [3]:
G = gList['0.txt']['graph']
y = torch.tensor(list(gList['0.txt']['score'].values()))

In [4]:
# Prepare nodes initial feature X [dv,1,1]
def gen_nodes_feature(G):
    deg = np.array(list(dict(sorted(dict(G.degree()).items())).values()))
    X = np.ones((3,len(deg)))
    X[0,:]=deg
    norms = np.linalg.norm(X,axis = 1,keepdims=True)
    X = torch.FloatTensor(X.T)
    return X

In [5]:
class My_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        return x, y

## 3a. DrBC encoder function

In [6]:
class DrBCEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers,G):
        super(DrBCEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.layer1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.norm1 = nn.BatchNorm1d(hidden_size)
        self.gru_cell = nn.GRUCell(hidden_size, hidden_size,bias = False)
        self.norm2 = nn.BatchNorm1d(hidden_size)
        self.G = G
        self.deg = dict(self.G.degree())
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.norm1(x)
        output = [x]
        for i in range(self.num_layers-1):
            hn = self.calHn(x)
            x = self.gru_cell(x,hn)
            x = self.norm2(x)
            output.append(x)
        output, _ = torch.max(torch.stack(output), dim=0)
        return output

    def calHn(self,x):
        hn = torch.zeros(x.shape).to(self.device)
        for node in self.G.nodes():
            degv = self.deg[node]
            for neigh in list(self.G.adj[node]):
                denominator = 1/(math.sqrt(degv+1)*math.sqrt(self.deg[neigh]+1))
                hn[node,:] += (denominator*x[neigh])
        return hn
    
'''
# Define the model
input_size = 3
hidden_size = 32
num_layers = 5
encoder = DrBCEncoder(input_size, hidden_size, num_layers,g)
X = gen_nodes_feature(g)
out = encoder(X)
print(out.shape)
print(out)
'''

'\n# Define the model\ninput_size = 3\nhidden_size = 32\nnum_layers = 5\nencoder = DrBCEncoder(input_size, hidden_size, num_layers,g)\nX = gen_nodes_feature(g)\nout = encoder(X)\nprint(out.shape)\nprint(out)\n'

## 3b. Decoder: 2-layer MLP

In [7]:
class DrBCDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DrBCDecoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Define the layers of the decoder
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.norm1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.norm2 = nn.BatchNorm1d(output_size)
        
    def forward(self, x):
        # Pass the input through the layers of the decoder
        x = self.layer1(x)
        x = self.norm1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.norm2(x)
        return x

## 3c. Training

In [8]:
def sampling(minimum,maximum,qty):
    pairs = []
    for i in range(qty):
        a = random.randint(minimum,maximum)
        b = random.randint(minimum,maximum)
        pairs.append((a,b))
    return pairs

In [9]:
def bc_pairs(pairs, outputs, y):
    pred = []
    gt = []
    g = nn.Sigmoid()
    for i, pair in enumerate(pairs):
        pred.append(g(outputs[pair[0]] - outputs[pair[1]]))
        gt.append(g(y[pair[0]] - y[pair[1]]))
    return torch.stack(pred), torch.stack(gt)

In [10]:
'''
G = gList['0.txt']['graph']
y = torch.tensor([list(gList['0.txt']['score'].values())])
y = torch.transpose(y,0,1)
# Define the models
input_size = 3
hidden_size = 128
output_size = 1
num_layers = 5
encoder = DrBCEncoder(input_size, hidden_size, num_layers,G)
decoder = DrBCDecoder(hidden_size,hidden_size,output_size)

n = G.number_of_nodes()
num_episodes = 20
lr = 0.001
sample_qty = 5*n

# Define the loss and optimizer
criterion = nn.BCELoss(reduction = 'sum')
optimizer = optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=lr)

# Get the inputs
inputs = gen_nodes_feature(G)

# Train the model
for episode in range(num_episodes):
    # model
    outputs = encoder(inputs)
    outputs = decoder(outputs)
    
    pairs = sampling(0,n-1,sample_qty)
    pred,gt = bc_pairs(pairs,outputs,y)
    loss = criterion(pred,gt)

    if ~loss.requires_grad:
        loss.requires_grad_()
        
    # Zero the parameter gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print statistics
    print('[%d] loss: %.4f' %(episode + 1, loss.item()))
'''

"\nG = gList['0.txt']['graph']\ny = torch.tensor([list(gList['0.txt']['score'].values())])\ny = torch.transpose(y,0,1)\n# Define the models\ninput_size = 3\nhidden_size = 128\noutput_size = 1\nnum_layers = 5\nencoder = DrBCEncoder(input_size, hidden_size, num_layers,G)\ndecoder = DrBCDecoder(hidden_size,hidden_size,output_size)\n\nn = G.number_of_nodes()\nnum_episodes = 20\nlr = 0.001\nsample_qty = 5*n\n\n# Define the loss and optimizer\ncriterion = nn.BCELoss(reduction = 'sum')\noptimizer = optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=lr)\n\n# Get the inputs\ninputs = gen_nodes_feature(G)\n\n# Train the model\nfor episode in range(num_episodes):\n    # model\n    outputs = encoder(inputs)\n    outputs = decoder(outputs)\n    \n    pairs = sampling(0,n-1,sample_qty)\n    pred,gt = bc_pairs(pairs,outputs,y)\n    loss = criterion(pred,gt)\n\n    if ~loss.requires_grad:\n        loss.requires_grad_()\n        \n    # Zero the parameter gradients\n    optimizer.zero

# 4. Evaluation Metric

## 4a. Top-N% accuracy

In [11]:
def topN(n,pred,gt):
    k = math.ceil(pred.size()[0]*n/100)
    _,pred_top = torch.topk(pred.view(-1),k=k)
    _,gt_top = torch.topk(gt.view(-1),k=k)
    intersect = torch.unique(torch.cat((pred_top,gt_top),0))
    acc = (2*k-len(intersect))/k
    return acc

## 4b. Kendall tau distance

In [12]:
def kendall(pred,gt):
    pred_ind = torch.argsort(pred)
    gt_ind = torch.argsort(gt)
    con = 0 # number of concordant pairs
    dcor = 0 # number of discordant pairs
    n = len(pred_ind)
    for i in range(n):
        if pred_ind[i] == gt_ind[i]:
            con += 1
        else:
            dcor += 1
    ken = 2*(con-dcor)/(n*(n-1))
    return ken

## 4c. Wall-clock running time

In [13]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        print(f"Running {func.__name__} ...", end='\r')
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} Done in {end - start:.2f} seconds")
        return result
    return wrapper

# 5. Putting all things together

In [14]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [15]:
def train_and_result(filename):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    k = [1,5,10]
    
    print("Working on {}".format(filename))
    # Prepare the data
    G = gList[filename]['graph']
    y = torch.tensor([list(gList[filename]['score'].values())])
    y = torch.transpose(y,0,1)

    # Define the models
    input_size = 3
    hidden_size = 128
    output_size = 1
    num_layers = 5
    encoder = DrBCEncoder(input_size, hidden_size, num_layers,G)
    decoder = DrBCDecoder(hidden_size,hidden_size,output_size)
    encoder.to(device)
    decoder.to(device)
    model = EncoderDecoder(encoder,decoder)
    model.to(device)
    
    n = G.number_of_nodes()
    num_episodes = 20
    lr = 0.001
    sample_qty = 5*n

    # Define the loss and optimizer
    criterion = nn.BCELoss(reduction = 'sum')
    #criterion = nn.BCEWithLogitsLoss(reduction = 'sum') # Because only pred will go through sigmoid, but ground truth won't
    #optimizer = optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=lr)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Get the inputs
    inputs = gen_nodes_feature(G)
    
    best_loss = float('inf')
    best_model_weights = None
    
    inputs = inputs.to(device)
    y = y.to(device)
    
    # Train the model
    print("Start training on {}".format(device))
    pairs = sampling(0,n-1,sample_qty)
    for episode in range(num_episodes):
        # Zero the parameter gradients
        optimizer.zero_grad()
    
        # model
        outputs = model(inputs)
        pred,gt = bc_pairs(pairs,outputs,y)
        loss = criterion(pred,gt)
        if ~loss.requires_grad:
            loss.requires_grad_()

        loss.backward()
        optimizer.step()
        
        if best_loss>loss:
            best_loss = loss
            best_out = outputs
            #best_model_weights = model.state_dict()
            
        # Print statistics
        #if episode%5 == 4:
        print('[%d] loss: %.4f' %(episode + 1, loss.item()))
            
    #torch.save(best_model_weights, 'best_model.pth')
    top1 = topN(1,best_out,y)
    top5 = topN(5,best_out,y)
    top10 = topN(10,best_out,y)
    ken = kendall(best_out,y)
    end_time = time.time()
    elapsed_time = end_time-start_time
    print("Elapsed time: {:.2f} seconds".format(elapsed_time))
    return top1,top5,top10,ken,elapsed_time

In [20]:
top1_list = []
top5_list = []
top10_list = []
ken_list = []
elapsed_time_list = []
for filename in filenames[:-1]:
    top1,top5,top10,ken,elapsed_time = train_and_result(filename)
    top1_list.append(top1)
    top5_list.append(top5)
    top10_list.append(top10)
    ken_list.append(ken)
    elapsed_time_list.append(elapsed_time)

Working on 0.txt
Start training on cuda
[1] loss: 21299.3242
[2] loss: 20293.3203
[3] loss: 19834.7461
[4] loss: 20113.3145
[5] loss: 19897.2500
[6] loss: 19670.0391
[7] loss: 19622.2090
[8] loss: 19558.9668
[9] loss: 19447.5742
[10] loss: 19196.9609
[11] loss: 19075.3496
[12] loss: 19002.4883
[13] loss: 18924.7070
[14] loss: 18872.3555
[15] loss: 18871.8789
[16] loss: 18807.5664
[17] loss: 18825.5742
[18] loss: 18782.5781
[19] loss: 18779.6562
[20] loss: 18753.7617
Elapsed time: 688.62 seconds
Working on 1.txt
Start training on cuda
[1] loss: 21423.6211
[2] loss: 20592.6738
[3] loss: 20577.1133
[4] loss: 20718.1211
[5] loss: 20638.0352
[6] loss: 20539.4785
[7] loss: 20480.2754
[8] loss: 20426.3945
[9] loss: 20355.5352
[10] loss: 20243.2383
[11] loss: 20065.2578
[12] loss: 19572.7617
[13] loss: 19522.2227
[14] loss: 19483.7812
[15] loss: 19448.4453
[16] loss: 19326.6797
[17] loss: 19170.7754
[18] loss: 18969.4336
[19] loss: 18927.1445
[20] loss: 18865.3711
Elapsed time: 681.05 seconds


[8] loss: 19609.6055
[9] loss: 19525.5879
[10] loss: 19224.6172
[11] loss: 19155.1758
[12] loss: 19098.3730
[13] loss: 19051.4570
[14] loss: 18996.3438
[15] loss: 18942.3242
[16] loss: 18913.7852
[17] loss: 18905.1328
[18] loss: 18848.3184
[19] loss: 18674.0000
[20] loss: 18855.9531
Elapsed time: 678.45 seconds
Working on 24.txt
Start training on cuda
[1] loss: 22355.7891
[2] loss: 20697.1953
[3] loss: 20376.0547
[4] loss: 20254.7402
[5] loss: 20122.3398
[6] loss: 20013.2891
[7] loss: 19859.1797
[8] loss: 19737.7188
[9] loss: 19626.5586
[10] loss: 19518.3242
[11] loss: 19439.8691
[12] loss: 19196.6211
[13] loss: 19142.7285
[14] loss: 18794.9219
[15] loss: 18753.7695
[16] loss: 18490.3477
[17] loss: 18716.0508
[18] loss: 18713.5781
[19] loss: 18662.0117
[20] loss: 18415.9648
Elapsed time: 679.51 seconds
Working on 25.txt
Start training on cuda
[1] loss: 22402.2148
[2] loss: 21026.3711
[3] loss: 20166.7812
[4] loss: 19919.0273
[5] loss: 19628.1367
[6] loss: 19479.2676
[7] loss: 19385.668

In [21]:
# Calculate Mean for each evaluation metrics:
print("Top-1% accuracy: {:.2f}±{:.2f}".format(np.mean(np.array(top1_list)),np.std(np.array(top1_list))))
print("Top-5% accuracy: {:.2f}±{:.2f}".format(np.mean(np.array(top5_list)),np.std(np.array(top5_list))))
print("Top-10% accuracy: {:.2f}±{:.2f}".format(np.mean(np.array(top10_list)),np.std(np.array(top10_list))))
print("Kendall tau distance: {:.2f}±{:.2f}".format(np.mean(np.array(ken_list)),np.std(np.array(ken_list))))
print("Running time: {:.2f}±{:.2f}".format(np.mean(np.array(elapsed_time_list)),np.std(np.array(elapsed_time_list))))

Top-1% accuracy: 0.28±0.18
Top-5% accuracy: 0.19±0.09
Top-10% accuracy: 0.22±0.07
Kendall tau distance: 0.00±0.00
Running time: 671.10±17.12


In [16]:
def train_and_result_for_com(filename):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    k = [1,5,10]
    
    print("Working on {}".format(filename))
    # Prepare the data
    G = gList[filename]['graph']
    y = torch.tensor([list(gList[filename]['score'].values())])
    y = torch.transpose(y,0,1)

    # Define the models
    input_size = 3
    hidden_size = 3
    output_size = 1
    num_layers = 1
    encoder = DrBCEncoder(input_size, hidden_size, num_layers,G)
    decoder = DrBCDecoder(hidden_size,hidden_size,output_size)
    encoder.to(device)
    decoder.to(device)
    model = EncoderDecoder(encoder,decoder)
    model.to(device)
    
    n = G.number_of_nodes()
    num_episodes = 5
    lr = 0.001
    sample_qty = int(0.5*n) # Reduce to 1.5|N|

    # Define the loss and optimizer
    criterion = nn.BCELoss(reduction = 'mean') # Sum may leak
    #criterion = nn.BCEWithLogitsLoss(reduction = 'sum') # Because only pred will go through sigmoid, but ground truth won't
    #optimizer = optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=lr)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Get the inputs
    inputs = gen_nodes_feature(G)
    
    best_loss = float('inf')
    best_model_weights = None
    
    inputs = inputs.to(device)
    y = y.to(device)
    
    # Train the model
    print("Start training on {}".format(device))
    pairs = sampling(0,n-1,sample_qty)
    for episode in range(num_episodes):
        # Zero the parameter gradients
        optimizer.zero_grad()
    
        # model
        outputs = model(inputs)
        print("finish output from model")
        pred,gt = bc_pairs(pairs,outputs,y)
        loss = criterion(pred,gt)
        if ~loss.requires_grad:
            loss.requires_grad_()

        loss.backward()
        optimizer.step()
        
        if best_loss>loss:
            best_loss = loss
            best_out = outputs
            #best_model_weights = model.state_dict()
            
        # Print statistics
        #if episode%5 == 4:
        print('[%d] loss: %.4f' %(episode + 1, loss.item()))
            
    #torch.save(best_model_weights, 'best_model.pth')
    top1 = topN(1,best_out,y)
    top5 = topN(5,best_out,y)
    top10 = topN(10,best_out,y)
    ken = kendall(best_out,y)
    end_time = time.time()
    elapsed_time = end_time-start_time
    print("Elapsed time: {:.2f} seconds".format(elapsed_time))
    return top1,top5,top10,ken,elapsed_time

In [ ]:
com_top1,com_top5,com_top10,com_ken,com_elapsed_time = train_and_result(filenames[-1])
print("Top-1% accuracy: {:.2f}".format(com_top1))
print("Top-5% accuracy: {:.2f}".format(com_top5))
print("Top-10% accuracy: {:.2f}".format(com_top10))
print("Kendall tau distance: {:.2f}".format(com_ken))
print("Running time: {:.2f}".format(com_elapsed_time))

Working on com-youtube.txt
Start training on cuda
